<a href="https://colab.research.google.com/github/moreirarodrigo-eo/Project_NSF/blob/main/Probit_regression_GEE_data_aquisition_no_subsets%2Bmonthly_subsets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=E_EAze0p03cRrlPdvdI1eIkyaQnjdBU1YiSH5MCIlzc&tc=i37m7fcmN2tdRc-ewd8eiNC_bsVafO17fiQIfCNMqiM&cc=Ycd-iCX6ouLjUpczwW6M1kDwlZjMOwiUoyTwNXrKnfg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmwYGJJcP09oT8BOVqMd9B0YKJACZzKdHiMpe0HAEcNuS4mO5OEQRM

Successfully saved authorization token.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install geemap -q
import geemap
import folium
!pip install geopandas -q
import geopandas as gpd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive/PROJETOS/Project_US/')

# **Monthly subsets - all months separatelly**

###**Loop**



In [7]:
lots = ee.FeatureCollection('projects/ee-moreirarmt/assets/Vectors/with_regionTotal_Lots_NewID').filter(ee.Filter.eq('Region', 3))
grid = ee.FeatureCollection.randomPoints(region = lots, points = 50000, seed = 42, maxError = 1)
grid_small = ee.FeatureCollection.randomPoints(region = lots, points = 10000, seed = 42, maxError = 1).limit(5)

# Define the parameters
regions = [
          # 'North',
           'South',
          #  'Center'
           ]
distance_month_subsets = [
                          50,
                          100,
                          150,
                          200
                          ]
months = [
          '_JUL_',
          '_AUG_',
          '_SEP_'
          ]
years = [
         '2019',
         '2020',
         '2021',
         '2022',
         '2023'
         ]

# Create a loop to iterate over all combinations
for region in regions:
    for distance_month_subset in distance_month_subsets:
        for month in months:
            for year in years:
                # Load the 'land_use' image for the specified year
                land_use = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1') \
                    .select(f'classification_2022').clip(lots)

                # Create a kernel and perform operations
                distance_kernel = 2550
                kernel = ee.Kernel.euclidean(distance_kernel, 'meters')

                # Example of calculating distances
                edges_land_use = ee.Algorithms.CannyEdgeDetector(land_use, 0)
                edges_land_use_reclass = edges_land_use.gt(0).clip(lots)

                # Calculate distances for land use edges
                edges_land_use_reclass_distance = edges_land_use_reclass.distance(kernel, False).lte(distance_month_subset).clip(lots)

                # Load 'roads' feature collection and calculate distances
                roads_ro = ee.FeatureCollection("projects/ee-moreirarmt/assets/Vectors/Roads_RO")
                roads_distance = roads_ro.distance(distance_kernel)
                roads_dist_reclass = ee.Image(roads_distance).lte(distance_month_subset).clip(lots)

                # Load 'lots_edges' feature collection and calculate distances
                lots_edges = ee.FeatureCollection('users/moreirarmt/Total_Lots_NewID')
                lots_edges_distance = lots_edges.distance(searchRadius=distance_kernel, maxError=1).lte(distance_month_subset).clip(lots)

                # Load and process 'burn_scars' image
                merge_burn_scars = ee.Image(f'projects/ee-moreirarmt/assets/FIRE_Project_US_Consult/PROJ_US_RF{month}{year}')
                burn_scars_edges = ee.Algorithms.CannyEdgeDetector(merge_burn_scars, 0).gt(0)
                merge_burn_scars_reclass = ee.Image(1).where(merge_burn_scars.gte(1).And(merge_burn_scars.lte(4)), 1)

                # Create a kernel for burn scars edges and calculate distances
                kernel_burn_scars = ee.Kernel.euclidean(distance_kernel, 'meters')
                burn_scars_edges_distance = burn_scars_edges.distance(kernel_burn_scars).lte(distance_month_subset).clip(lots)

                # Combine and rename image bands
                band_names = [
                    f'roads_distance_{distance_month_subset}_{month}_{region}',
                    f'land_use_edge_distance_{distance_month_subset}_{month}_{region}',
                    f'burn_scars_edges_distance_{distance_month_subset}_{month}_{region}',
                    f'lots_edges_distance_{distance_month_subset}_{month}_{region}'
                ]
                images_combined = roads_dist_reclass.addBands(edges_land_use_reclass_distance) \
                                    .addBands(burn_scars_edges_distance) \
                                    .addBands(lots_edges_distance) \
                                    .rename(band_names)

                # Sample regions and create a Pandas DataFrame
                extracted_values = images_combined.sampleRegions(collection=grid,
                                                                 scale=10)
                grid_df = geemap.ee_to_pandas(grid_small)
                sample_result = extracted_values.first().getInfo()
                columns = list(sample_result['properties'].keys())
                column_df = list(grid_df.columns)
                column_names = [
                    f'burn_scars_edges_distance_{distance_month_subset}_{month}_{region}',
                    f'land_use_edge_distance_{distance_month_subset}_{month}_{region}',
                    f'roads_distance_{distance_month_subset}_{month}_{region}',
                    f'lots_edges_distance_{distance_month_subset}_{month}_{region}'
                ]
                column_df.extend(column_names)
                nested_list = extracted_values.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
                data = nested_list.getInfo()
                df = pd.DataFrame(data, columns=column_df)
                pd.DataFrame.to_csv(df, f'monthly_subsets_LOOP/{region}/{year}/extract_values_covariates_burn_edges_{distance_month_subset}_m_df{month}{year}.csv')

In [ ]:
import geopandas as gpd
import pandas as pd
import os

regions = ['North', 'South', 'Center']
distance_month_subsets = [50, 100, 150, 200]
months = ['_JUL_', '_AUG_', '_SEP_']
years = ['2019', '2020', '2021', '2022']

# grid_gdf = gpd.read_file('grid_points_probit.shp')

dataframes = {}  # Create a dictionary to store merged DataFrames
filtered_dataframes = {}  # Create a dictionary to store filtered DataFrames

for region in regions:
    for distance_month_subset in distance_month_subsets:
        for month in months:
            for year in years:
                file_path = f'monthly_subsets_LOOP/{region}/{year}/extract_values_covariates_burn_edges_{distance_month_subset}_m_df{month}{year}.csv'
                df = pd.read_csv(file_path)

                # Rename 'Unnamed: 0' column to 'id'
                df = df.rename(columns={'Unnamed: 0': 'id'})

                # Ensure both DataFrames have a common column to merge on (e.g., 'geometry' if it exists)
                merge_on = 'id' if 'id' in df.columns else 'common_column_name'

                merged_df = grid_gdf.merge(df, on=merge_on)
                merged_df = merged_df.drop(columns=['id'])  # Remove unnecessary 'id' column
                key = f'db_{region}_{distance_month_subset}m_{month}_{year}'
                dataframes[key] = merged_df

                filtered_df = merged_df[merged_df[f'burn_scars_edges_distance_{distance_month_subset}_{month}_{region}'] > 0]
                filtered_dataframes[key] = filtered_df

                merged_df['lon'] = merged_df['geometry'].x
                merged_df['lat'] = merged_df['geometry'].y

                output_path_csv = f'PROBIT_spreg_csv_lat_long_LOOP/{region}/{year}/{distance_month_subset}/{distance_month_subset}_m_df{month}{year}.csv'
                merged_df.to_csv(output_path_csv)

                output_path_shp = f'PROBIT_spreg_csv_lat_long_LOOP/{region}/{year}/{distance_month_subset}/{distance_month_subset}_m_df{month}{year}.shp'
                merged_df.to_file(output_path_shp)

<ipython-input-25-e211607660f8>:43: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_df.to_file(output_path_shp)
<ipython-input-25-e211607660f8>:43: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_df.to_file(output_path_shp)
<ipython-input-25-e211607660f8>:43: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_df.to_file(output_path_shp)
<ipython-input-25-e211607660f8>:43: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_df.to_file(output_path_shp)
<ipython-input-25-e211607660f8>:43: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_df.to_file(output_path_shp)
<ipython-input-25-e211607660f8>:43: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.

###**Monthly subsets**


In [5]:
lots = ee.FeatureCollection('projects/ee-moreirarmt/assets/Vectors/with_regionTotal_Lots_NewID').filter(ee.Filter.eq('Region', 1))
grid = ee.FeatureCollection('projects/ee-moreirarmt/assets/rand_points_North_10000')
grid_small = ee.FeatureCollection('projects/ee-rodrigomoreira/assets/grid_pontos_Project_US')

distance_month_subset = 50
month = '_JUL_'
year = str(2019)

distance = distance_month_subset
distance_kernel = 2550
kernel = ee.Kernel.euclidean(distance_kernel, 'meters')

# ##Land use
# land_use = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1')\
#                 .select('classification_'+year).clip(lots)

##Land use for 2023
land_use = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1')\
                .select('classification_2022').clip(lots)

land_use = ee.Image(land_use)
edges_land_use = ee.Algorithms.CannyEdgeDetector(land_use, 0)
edges_land_use_reclass = edges_land_use.gt(0).clip(lots)
#edges_land_use.gte(0).And(edges_land_use.lte(distance), 1).where(edges_land_use.gt(distance + 0.1).And(edges_land_use.lte(distance_kernel), 0)),
edges_land_use_reclass_distance  = edges_land_use_reclass.distance(kernel, False).lte(distance).clip(lots)


##Roads
roads_ro = ee.FeatureCollection("projects/ee-moreirarmt/assets/Vectors/Roads_RO")
roads_distance = roads_ro.distance(distance_kernel)
roads_dist_reclass = ee.Image(roads_distance).lte(distance).clip(lots)

##Lots
lots_edges = ee.FeatureCollection('users/moreirarmt/Total_Lots_NewID')
lots_edges_distance = lots_edges.distance(searchRadius = distance_kernel, maxError = 1).lte(distance).clip(lots)

merge_burn_scars = ee.Image('projects/ee-moreirarmt/assets/FIRE_Project_US_Consult/PROJ_US_RF'+month+year)

burn_scars_edges = ee.Algorithms.CannyEdgeDetector(merge_burn_scars,0).gt(0)

merge_burn_scars_reclass = ee.Image(1)\
                            .where(merge_burn_scars.gte(1).And(merge_burn_scars.lte(4)), 1);

kernel_burn_scars = ee.Kernel.euclidean(distance_kernel, 'meters')

burn_scars_edges_distance = burn_scars_edges.distance(kernel_burn_scars).lte(distance).clip(lots)

burn_scars_edges_within_excluded = merge_burn_scars_reclass.add(burn_scars_edges_distance).eq(2)

burn_scars_edges_within_excluded_reclass_to_2 = burn_scars_edges_within_excluded\
                                                    .where(merge_burn_scars.gt(0), 2)

images_combined = roads_dist_reclass.addBands(edges_land_use_reclass_distance)\
                                    .addBands(burn_scars_edges_within_excluded_reclass_to_2)\
                                    .addBands(lots_edges_distance)\
                                    .rename(['roads_distance_'+month, 'land_use_edge_distance_'+month,
                                             'burn_scars_edges_distance_'+month, 'lots_edges_distance_'+month])

extracted_values = images_combined.sampleRegions(collection = grid,
                                                 scale = 10)
grid_df = geemap.ee_to_pandas(grid_small)

sample_result = extracted_values.first().getInfo()

columns = list(sample_result['properties'].keys())
column_df = list(grid_df.columns)
column_df.extend(['burn_scars_edges_distance_'+month, 'land_use_edge_distance_'+month, 'roads_distance_'+month, 'lots_edges_distance_'+month])

nested_list = extracted_values.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
data = nested_list.getInfo()
df = pd.DataFrame(data, columns = column_df)
pd.DataFrame.to_csv(df, 'monthly_subsets/'+str(year)+'/extract_values_covariates_burn_edges_'+str(distance_month_subset)+'_m_df'+month+str(year)+'.csv')

###**Monthly+Region subsets**


In [ ]:
lots = ee.FeatureCollection('projects/ee-moreirarmt/assets/Vectors/with_regionTotal_Lots_NewID').filter(ee.Filter.eq('Region', 1))
grid = ee.FeatureCollection.randomPoints(region = lots, points = 50000, seed = 42, maxError = 1)
grid_small = ee.FeatureCollection.randomPoints(region = lots, points = 2, seed = 42, maxError = 1)
region = 'North'
distance_month_subset = 50
month = '_JUL_'
year = str(2019)

distance = distance_month_subset
distance_kernel = 500
kernel = ee.Kernel.euclidean(distance_kernel, 'meters')

##Land use
land_use = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1')\
                .select('classification_'+year).clip(lots)

# ##Land use for 2023
# land_use = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1')\
#                 .select('classification_2022').clip(lots)

land_use = ee.Image(land_use)
edges_land_use = ee.Algorithms.CannyEdgeDetector(land_use, 0)
edges_land_use_reclass = edges_land_use.gt(0).clip(lots)
#edges_land_use.gte(0).And(edges_land_use.lte(distance), 1).where(edges_land_use.gt(distance + 0.1).And(edges_land_use.lte(distance_kernel), 0)),
edges_land_use_reclass_distance  = edges_land_use_reclass.distance(kernel, False).lte(distance).clip(lots)


##Roads
roads_ro = ee.FeatureCollection("projects/ee-moreirarmt/assets/Vectors/Roads_RO")
roads_distance = roads_ro.distance(distance_kernel)
roads_dist_reclass = ee.Image(roads_distance).lte(distance).clip(lots)

##Lots
lots_edges = ee.FeatureCollection('users/moreirarmt/Total_Lots_NewID')
lots_edges_distance = lots_edges.distance(searchRadius = distance_kernel, maxError = 1).lte(distance).clip(lots)

merge_burn_scars = ee.Image('projects/ee-moreirarmt/assets/FIRE_Project_US_Consult/PROJ_US_RF'+month+year)

burn_scars_edges = ee.Algorithms.CannyEdgeDetector(merge_burn_scars,0).gt(0)

merge_burn_scars_reclass = ee.Image(1)\
                            .where(merge_burn_scars.gte(1).And(merge_burn_scars.lte(4)), 1);

kernel_burn_scars = ee.Kernel.euclidean(distance, 'meters')

burn_scars_edges_distance = burn_scars_edges.distance(kernel_burn_scars).lte(distance).clip(lots)

burn_scars_edges_within_excluded = merge_burn_scars_reclass.add(burn_scars_edges_distance).eq(2)

burn_scars_edges_within_excluded_reclass_to_2 = burn_scars_edges_within_excluded\
                                                    .where(merge_burn_scars.gt(0), 2)

images_combined = roads_dist_reclass.addBands(edges_land_use_reclass_distance)\
                                    .addBands(burn_scars_edges_within_excluded_reclass_to_2)\
                                    .addBands(lots_edges_distance)\
                                    .rename(['roads_distance_'+month+'_'+region, 'land_use_edge_distance_'+month+'_'+region,
                                             'burn_scars_edges_distance_'+month+'_'+region, 'lots_edges_distance_'+month+'_'+region])


extracted_values = images_combined.sampleRegions(collection = grid,
                                                 scale = 10,
                                                 tileScale = 8)
grid_df = geemap.ee_to_pandas(grid_small)

sample_result = extracted_values.first().getInfo()

columns = list(sample_result['properties'].keys())
column_df = list(grid_df.columns)
column_df.extend(['burn_scars_edges_distance_'+month+'_'+region, 'land_use_edge_distance_'+month+'_'+region, 'roads_distance_'+month+'_'+region, 'lots_edges_distance_'+month+'_'+region])

nested_list = extracted_values.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
data = nested_list.getInfo()
df = pd.DataFrame(data, columns = column_df)
pd.DataFrame.to_csv(df, 'monthly_subsets/'+region+'/'+str(year)+'/extract_values_covariates_burn_edges_'+str(distance_month_subset)+'_m_df'+month+str(year)+'.csv')

###**Exporting as csv, creating gdfs subsets**


In [ ]:
distance_month_subset_50 = 50
distance_month_subset_100 = 100
distance_month_subset_150 = 150
distance_month_subset_200 = 200

month = 'SEP'
year = str(2023)

df_50m = pd.read_csv('monthly_subsets/'+str(year)+'/extract_values_covariates_burn_edges_'+str(distance_month_subset_50)+'_m_df_'+month+'_'+str(year)+'.csv')
df_100m =pd.read_csv('monthly_subsets/'+str(year)+'/extract_values_covariates_burn_edges_'+str(distance_month_subset_100)+'_m_df_'+month+'_'+str(year)+'.csv')
df_150m = pd.read_csv('monthly_subsets/'+str(year)+'/extract_values_covariates_burn_edges_'+str(distance_month_subset_150)+'_m_df_'+month+'_'+str(year)+'.csv')
df_200m = pd.read_csv('monthly_subsets/'+str(year)+'/extract_values_covariates_burn_edges_'+str(distance_month_subset_200)+'_m_df_'+month+'_'+str(year)+'.csv')

In [ ]:
grid_gdf = gpd.read_file('grid_points_probit.shp')

db_50m = grid_gdf.merge(df_50m, on = 'id')
db_50m = db_50m.drop(columns = ['id','Unnamed: 0'])

db_100m = grid_gdf.merge(df_100m, on = 'id')
db_100m = db_100m.drop(columns = ['id','Unnamed: 0'])

db_150m = grid_gdf.merge(df_150m, on = 'id')
db_150m = db_150m.drop(columns = ['id','Unnamed: 0'])

db_200m = grid_gdf.merge(df_200m, on = 'id')
db_200m = db_200m.drop(columns = ['id','Unnamed: 0'])

In [ ]:
db_200m

,geometry,burn_scars_edges_distance__SEP_,land_use_edge_distance__SEP_,roads_distance__SEP_,lots_edges_distance__SEP_
0,POINT (-63.23647 -9.90487),0,1,0,1
1,POINT (-63.14906 -10.02297),0,1,0,0
2,POINT (-62.53559 -11.05455),0,0,0,1
3,POINT (-62.22685 -10.63039),0,0,1,1
4,POINT (-61.56292 -12.42399),0,1,0,1
...,...,...,...,...,...
32944,POINT (-61.90157 -10.31213),0,0,1,0
32945,POINT (-61.57247 -12.16362),0,1,0,1
32946,POINT (-61.95993 -10.13006),0,1,0,0
32947,POINT (-62.94059 -10.29249),0,1,0,0


In [ ]:
db_50m_ = db_50m[db_50m['burn_scars_edges_distance__'+month+'_'] > 0]
db_100m_ = db_100m[db_100m['burn_scars_edges_distance__'+month+'_'] > 0]
db_150m_ = db_150m[db_150m['burn_scars_edges_distance__'+month+'_'] > 0]
db_200m_ = db_200m[db_200m['burn_scars_edges_distance__'+month+'_'] > 0]

In [ ]:
db_50m_['lon'] = db_50m_['geometry'].x
db_50m_['lat'] = db_50m_['geometry'].y

db_100m_['lon'] = db_100m_['geometry'].x
db_100m_['lat'] = db_100m_['geometry'].y

db_150m_['lon'] = db_150m_['geometry'].x
db_150m_['lat'] = db_150m_['geometry'].y

db_200m_['lon'] = db_200m_['geometry'].x
db_200m_['lat'] = db_200m_['geometry'].y

In [ ]:
db_50m_.to_csv('PROBIT_spreg_csv_lat_long/'+str(year)+'/'+str(distance_month_subset_50)+'_m_df'+month+str(year)+'.csv')
db_100m_.to_csv('PROBIT_spreg_csv_lat_long/'+str(year)+'/'+str(distance_month_subset_100)+'_m_df'+month+str(year)+'.csv')
db_150m_.to_csv('PROBIT_spreg_csv_lat_long/'+str(year)+'/'+str(distance_month_subset_150)+'_m_df'+month+str(year)+'.csv')
db_200m_.to_csv('PROBIT_spreg_csv_lat_long/'+str(year)+'/'+str(distance_month_subset_200)+'_m_df'+month+str(year)+'.csv')

In [ ]:
db_50m_.to_file('PROBIT_spreg_csv_lat_long/'+str(year)+'/'+str(distance_month_subset_50)+'_m_df'+month+str(year)+'.shp')
db_100m_.to_file('PROBIT_spreg_csv_lat_long/'+str(year)+'/'+str(distance_month_subset_100)+'_m_df'+month+str(year)+'.shp')
db_150m_.to_file('PROBIT_spreg_csv_lat_long/'+str(year)+'/'+str(distance_month_subset_150)+'_m_df'+month+str(year)+'.shp')
db_200m_.to_file('PROBIT_spreg_csv_lat_long/'+str(year)+'/'+str(distance_month_subset_200)+'_m_df'+month+str(year)+'.shp')

## **Monthly subsets - all jul, aug, and sep**

In [ ]:
lots = ee.FeatureCollection('users/moreirarmt/Total_Lots_NewID_dissolve')
grid = ee.FeatureCollection('projects/ee-rodrigomoreira/assets/grid_pontos_Project_US_100000')
grid_small = ee.FeatureCollection('projects/ee-rodrigomoreira/assets/grid_pontos_Project_US')

distance_month_subset = 50
distance = distance_month_subset
distance_kernel = 15000
kernel = ee.Kernel.euclidean(distance_kernel, 'meters')
year = str(2020)


##Land use
land_use = ee.Image('projects/mapbiomas-workspace/public/collection7/mapbiomas_collection70_integration_v2')\
                .select('classification_'+year).clip(lots)
land_use = ee.Image(land_use)
edges_land_use = ee.Algorithms.CannyEdgeDetector(land_use, 0)
edges_land_use_reclass = edges_land_use.gt(0).clip(lots)
#edges_land_use.gte(0).And(edges_land_use.lte(distance), 1).where(edges_land_use.gt(distance + 0.1).And(edges_land_use.lte(distance_kernel), 0)),
edges_land_use_reclass_distance  = edges_land_use_reclass.distance(kernel, False).lte(distance).clip(lots)


##Roads
roads_ro = ee.FeatureCollection("projects/ee-moreirarmt/assets/Roads_RO")
roads_distance = roads_ro.distance(distance_kernel)
roads_dist_reclass = ee.Image(roads_distance).lte(distance).clip(lots)

##Lots
lots_edges = ee.FeatureCollection('users/moreirarmt/Total_Lots_NewID')
lots_edges_distance = lots_edges.distance(searchRadius = distance_kernel, maxError = 1).lte(distance).clip(lots)


##Burn scars
burn_scars_JUL_2019 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_JUL_2019_focal_5')
burn_scars_AUG_2019 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_AUG_2019_focal_5')
burn_scars_SEP_2019 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_SEP_2019_focal_5')
burn_scars_JUL_2020 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_JUL_2020_focal_5')
burn_scars_AUG_2020 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_AUG_2020_focal_5')
burn_scars_SEP_2020 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_SEP_2020_focal_5')
burn_scars_JUL_2021 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_JUL_2021_focal_5')
burn_scars_AUG_2021 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_AUG_2021_focal_5')
burn_scars_SEP_2021 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_SEP_2021_focal_5')

In [ ]:
merge_burn_scars_jul = burn_scars_JUL_2019.add(burn_scars_JUL_2020).add(burn_scars_JUL_2021)


burn_scars_edges_jul = ee.Algorithms.CannyEdgeDetector(merge_burn_scars_jul,0).gt(0)

merge_burn_scars_reclass_jul = ee.Image(1)\
                            .where(merge_burn_scars_jul.gte(1).And(merge_burn_scars_jul.lte(4)), 1);

kernel_burn_scars_jul = ee.Kernel.euclidean(5000, 'meters')

burn_scars_edges_distance_jul = burn_scars_edges_jul.distance(kernel_burn_scars_jul).lte(distance).clip(lots)

burn_scars_edges_within_excluded_jul = merge_burn_scars_reclass_jul.add(burn_scars_edges_distance_jul).eq(2)

burn_scars_edges_within_excluded_reclass_to_2_jul = burn_scars_edges_within_excluded_jul\
                                                    .where(merge_burn_scars_jul.gt(0), 2)

images_combined_jul = roads_dist_reclass.addBands(edges_land_use_reclass_distance)\
                                    .addBands(burn_scars_edges_within_excluded_reclass_to_2_jul)\
                                    .addBands(lots_edges_distance)\
                                    .rename(['roads_distance', 'land_use_edge_distance',
                                             'burn_scars_edges_distance_jul', 'lots_edges_distance'])


extracted_values_jul = images_combined_jul.sampleRegions(collection = grid,
                                                 scale = 60)
grid_df_jul = geemap.ee_to_pandas(grid_small)

sample_result_jul = extracted_values_jul.first().getInfo()

columns_jul = list(sample_result_jul['properties'].keys())
column_df_jul = list(grid_df_jul.columns)
column_df_jul.extend(['burn_scars_edges_distance_jul', 'land_use_edge_distance', 'roads_distance', 'lots_edges_distance'])

nested_list_jul = extracted_values_jul.reduceColumns(ee.Reducer.toList(len(column_df_jul)), column_df_jul).values().get(0)
data_jul = nested_list_jul.getInfo()
data_jul
df_jul = pd.DataFrame(data_jul, columns = column_df_jul)
pd.DataFrame.to_csv(df_jul, 'extract_values_covariates_burn_edges_'+str(distance_month_subset)+'_m_df_JUL.csv')

In [ ]:
merge_burn_scars_aug = burn_scars_AUG_2019.add(burn_scars_AUG_2020).add(burn_scars_AUG_2021)


burn_scars_edges_aug = ee.Algorithms.CannyEdgeDetector(merge_burn_scars_aug,0).gt(0)

merge_burn_scars_reclass_aug = ee.Image(1)\
                            .where(merge_burn_scars_aug.gte(1).And(merge_burn_scars_aug.lte(4)), 1);

kernel_burn_scars_aug = ee.Kernel.euclidean(5000, 'meters')

burn_scars_edges_distance_aug = burn_scars_edges_aug.distance(kernel_burn_scars_aug).lte(distance).clip(lots)

burn_scars_edges_within_excluded_aug = merge_burn_scars_reclass_aug.add(burn_scars_edges_distance_aug).eq(2)

burn_scars_edges_within_excluded_reclass_to_2_aug = burn_scars_edges_within_excluded_aug\
                                                    .where(merge_burn_scars_aug.gt(0), 2)

images_combined_aug = roads_dist_reclass.addBands(edges_land_use_reclass_distance)\
                                    .addBands(burn_scars_edges_within_excluded_reclass_to_2_aug)\
                                    .addBands(lots_edges_distance)\
                                    .rename(['roads_distance', 'land_use_edge_distance',
                                             'burn_scars_edges_distance_aug', 'lots_edges_distance'])


extracted_values_aug = images_combined_aug.sampleRegions(collection = grid,
                                                 scale = 60)
grid_df_aug = geemap.ee_to_pandas(grid_small)

sample_result_aug = extracted_values_aug.first().getInfo()

columns_aug = list(sample_result_aug['properties'].keys())
column_df_aug = list(grid_df_aug.columns)
column_df_aug.extend(['burn_scars_edges_distance_aug', 'land_use_edge_distance', 'roads_distance', 'lots_edges_distance'])

nested_list_aug = extracted_values_aug.reduceColumns(ee.Reducer.toList(len(column_df_aug)), column_df_aug).values().get(0)
data_aug = nested_list_aug.getInfo()
data_aug
df_aug = pd.DataFrame(data_aug, columns = column_df_aug)
pd.DataFrame.to_csv(df_aug, 'extract_values_covariates_burn_edges_'+str(distance_month_subset)+'_m_df_AUG.csv')

In [ ]:
merge_burn_scars_sep = burn_scars_SEP_2019.add(burn_scars_SEP_2020).add(burn_scars_SEP_2021)


burn_scars_edges_sep = ee.Algorithms.CannyEdgeDetector(merge_burn_scars_sep,0).gt(0)

merge_burn_scars_reclass_sep = ee.Image(1)\
                            .where(merge_burn_scars_sep.gte(1).And(merge_burn_scars_sep.lte(4)), 1);

kernel_burn_scars_sep = ee.Kernel.euclidean(5000, 'meters')

burn_scars_edges_distance_sep = burn_scars_edges_sep.distance(kernel_burn_scars_sep).lte(distance).clip(lots)

burn_scars_edges_within_excluded_sep = merge_burn_scars_reclass_sep.add(burn_scars_edges_distance_sep).eq(2)

burn_scars_edges_within_excluded_reclass_to_2_sep = burn_scars_edges_within_excluded_sep\
                                                    .where(merge_burn_scars_sep.gt(0), 2)

images_combined_sep = roads_dist_reclass.addBands(edges_land_use_reclass_distance)\
                                    .addBands(burn_scars_edges_within_excluded_reclass_to_2_sep)\
                                    .addBands(lots_edges_distance)\
                                    .rename(['roads_distance', 'land_use_edge_distance',
                                             'burn_scars_edges_distance_sep', 'lots_edges_distance'])


extracted_values_sep = images_combined_sep.sampleRegions(collection = grid,
                                                 scale = 60)
grid_df_sep = geemap.ee_to_pandas(grid_small)

sample_result_sep = extracted_values_sep.first().getInfo()

columns_sep = list(sample_result_sep['properties'].keys())
column_df_sep = list(grid_df_sep.columns)
column_df_sep.extend(['burn_scars_edges_distance_sep', 'land_use_edge_distance', 'roads_distance', 'lots_edges_distance'])

nested_list_sep = extracted_values_sep.reduceColumns(ee.Reducer.toList(len(column_df_sep)), column_df_sep).values().get(0)
data_sep = nested_list_sep.getInfo()
data_sep
df_sep = pd.DataFrame(data_sep, columns = column_df_sep)
pd.DataFrame.to_csv(df_sep, 'extract_values_covariates_burn_edges_'+str(distance_month_subset)+'_m_df_SEP.csv')

## **Extract the raster data per points**


In [ ]:
lots = ee.FeatureCollection('users/moreirarmt/Total_Lots_NewID_dissolve')
grid = ee.FeatureCollection('projects/ee-rodrigomoreira/assets/grid_pontos_Project_US_100000')
grid_small = ee.FeatureCollection('projects/ee-rodrigomoreira/assets/grid_pontos_Project_US')


distance = 150
distance_kernel = 15000
kernel = ee.Kernel.euclidean(distance_kernel, 'meters')
year = str(2020)


##Land use
land_use = ee.Image('projects/mapbiomas-workspace/public/collection7/mapbiomas_collection70_integration_v2')\
                .select('classification_'+year).clip(lots)
land_use = ee.Image(land_use)
edges_land_use = ee.Algorithms.CannyEdgeDetector(land_use, 0)
edges_land_use_reclass = edges_land_use.gt(0).clip(lots)
#edges_land_use.gte(0).And(edges_land_use.lte(distance), 1).where(edges_land_use.gt(distance + 0.1).And(edges_land_use.lte(distance_kernel), 0)),
edges_land_use_reclass_distance  = edges_land_use_reclass.distance(kernel, False).lte(distance).clip(lots)


##Roads
roads_ro = ee.FeatureCollection("projects/ee-moreirarmt/assets/Roads_RO")
roads_distance = roads_ro.distance(distance_kernel)
roads_dist_reclass = ee.Image(roads_distance).lte(distance).clip(lots)


##Burn scars
burn_scars_JUL_2019 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_JUL_2019_focal_5')
burn_scars_AUG_2019 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_AUG_2019_focal_5')
burn_scars_SEP_2019 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_SEP_2019_focal_5')
burn_scars_JUL_2020 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_JUL_2020_focal_5')
burn_scars_AUG_2020 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_AUG_2020_focal_5')
burn_scars_SEP_2020 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_SEP_2020_focal_5')
burn_scars_JUL_2021 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_JUL_2021_focal_5')
burn_scars_AUG_2021 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_AUG_2021_focal_5')
burn_scars_SEP_2021 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_SEP_2021_focal_5')
burn_scars_JUL_2022 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_JUL_2022_focal_5')
burn_scars_AUG_2022 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_AUG_2022_focal_5')
burn_scars_SEP_2022 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_SEP_2022_focal_5')
# burn_scars_JUL_2023 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_JUL_2023_focal_5')
burn_scars_AUG_2023 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_AUG_2023_focal_5')
burn_scars_SEP_2023 = ee.Image('projects/ee-moreirarmt/assets/Project_US_focalmode_5/PROJ_US_RF_SEP_2023_focal_5')

merge_burn_scars = burn_scars_JUL_2019.add(burn_scars_AUG_2019).add(burn_scars_SEP_2019)\
                                .add(burn_scars_JUL_2020).add(burn_scars_AUG_2020).add(burn_scars_SEP_2020)\
                                .add(burn_scars_JUL_2021).add(burn_scars_AUG_2021).add(burn_scars_SEP_2021)\
                                .add(burn_scars_JUL_2022).add(burn_scars_AUG_2022).add(burn_scars_SEP_2022)\
                                # .add(burn_scars_JUL_2023)
                                .add(burn_scars_AUG_2023).add(burn_scars_SEP_2023)


burn_scars_edges = ee.Algorithms.CannyEdgeDetector(merge_burn_scars,0).gt(0)

merge_burn_scars_reclass = ee.Image(1)\
                            .where(merge_burn_scars.gte(1).And(merge_burn_scars.lte(4)), 1);

kernel_burn_scars = ee.Kernel.euclidean(5000, 'meters')

burn_scars_edges_distance = burn_scars_edges.distance(kernel_burn_scars).lte(distance).clip(lots)

burn_scars_edges_within_excluded = merge_burn_scars_reclass.add(burn_scars_edges_distance).eq(2)

burn_scars_edges_within_excluded_reclass_to_2 = burn_scars_edges_within_excluded\
                                                    .where(merge_burn_scars.gt(0), 2)

##Lots
lots_edges = ee.FeatureCollection('users/moreirarmt/Total_Lots_NewID')
lots_edges_distance = lots_edges.distance(searchRadius = distance_kernel, maxError = 1).lte(distance).clip(lots)

##Stack all images
images_combined = roads_dist_reclass.addBands(edges_land_use_reclass_distance)\
                                    .addBands(burn_scars_edges_within_excluded_reclass_to_2)\
                                    .addBands(lots_edges_distance)\
                                    .rename(['roads_distance', 'land_use_edge_distance',
                                             'burn_scars_edges_distance', 'lots_edges_distance'])

In [ ]:
extracted_values = images_combined.sampleRegions(collection = grid,
                                                 scale = 10)
print(extracted_values.first().getInfo())
grid_df = geemap.ee_to_pandas(grid_small)

{'type': 'Feature', 'geometry': None, 'id': '00000000000000000000_0', 'properties': {'burn_scars_edges_distance': 0, 'fid': 1, 'id': 0, 'land_use_edge_distance': 1, 'lots_edges_distance': 0, 'roads_distance': 0}}


In [ ]:
sample_result = extracted_values.first().getInfo()

columns = list(sample_result['properties'].keys())
column_df = list(grid_df.columns)
column_df.extend(['burn_scars_edges_distance', 'land_use_edge_distance', 'roads_distance', 'lots_edges_distance'])
print(column_df)

['id', 'burn_scars_edges_distance', 'land_use_edge_distance', 'roads_distance', 'lots_edges_distance']


In [ ]:
nested_list = extracted_values.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
data = nested_list.getInfo()
data
df_ = pd.DataFrame(data, columns = column_df)

In [ ]:
pd.DataFrame.to_csv(df_, 'extract_values_covariates_burn_edges_'+str(distance)+'_m.csv')

##**Preparing all geodataframes for PROBIT regression**

In [ ]:
!pip install geopandas
import geopandas as gpd
import pandas as pd

In [ ]:
grid_gdf = gpd.read_file('grid_points_probit.shp')
df_50m = pd.read_csv('extract_values_covariates_burn_edges_50_m.csv')
df_100m =pd.read_csv('extract_values_covariates_burn_edges_100_m.csv')
df_150m = pd.read_csv('extract_values_covariates_burn_edges_150_m.csv')
df_200m = pd.read_csv('extract_values_covariates_burn_edges_200_m.csv')

In [ ]:
db_50m = grid_gdf.merge(df_50m, on = 'id')
db_50m = db_50m.drop(columns = ['id','Unnamed: 0'])

db_100m = grid_gdf.merge(df_100m, on = 'id')
db_100m = db_100m.drop(columns = ['id','Unnamed: 0'])

db_150m = grid_gdf.merge(df_150m, on = 'id')
db_150m = db_150m.drop(columns = ['id','Unnamed: 0'])

db_200m = grid_gdf.merge(df_200m, on = 'id')
db_200m = db_200m.drop(columns = ['id','Unnamed: 0'])

In [ ]:
db_50m_ = db_50m[db_50m['burn_scars_edges_distance'] > 0]
db_100m_ = db_100m[db_100m['burn_scars_edges_distance'] > 0]
db_150m_ = db_150m[db_150m['burn_scars_edges_distance'] > 0]
db_200m_ = db_200m[db_200m['burn_scars_edges_distance'] > 0]

In [ ]:
db_50m_['lon'] = db_50m_['geometry'].x
db_50m_['lat'] = db_50m_['geometry'].y

db_100m_['lon'] = db_100m_['geometry'].x
db_100m_['lat'] = db_100m_['geometry'].y

db_150m_['lon'] = db_150m_['geometry'].x
db_150m_['lat'] = db_150m_['geometry'].y

db_200m_['lon'] = db_200m_['geometry'].x
db_200m_['lat'] = db_200m_['geometry'].y

In [ ]:
db_50m_.to_csv('PROBIT_spreg_csv_lat_long/db_50m.csv')
db_100m_.to_csv('PROBIT_spreg_csv_lat_long/db_100m.csv')
db_150m_.to_csv('PROBIT_spreg_csv_lat_long/db_150m.csv')
db_200m_.to_csv('PROBIT_spreg_csv_lat_long/db_200m.csv')

In [ ]:
db_50m_.to_file('PROBIT_spreg_db/db_50m.shp')
db_100m_.to_file('PROBIT_spreg_db/db_100m.shp')
db_150m_.to_file('PROBIT_spreg_db/db_150m.shp')
db_200m_.to_file('PROBIT_spreg_db/db_200m.shp')